In [19]:
import uproot3 as uproot
import uproot as uproot4
import numpy as np
import numba
import awkward as ak
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib import rcParams
import mplhep as hep
plt.style.use(hep.style.ROOT)
import os,sys
import pandas as pd
from scipy.stats import halfnorm
from sklearn.cluster import KMeans
import random
from scipy.stats import binned_statistic
from scipy.stats import binned_statistic_2d
from scipy.optimize import curve_fit
from scipy.stats import crystalball
from lmfit import Model
import math
from scipy.stats import halfnorm
#from sklearn.cluster import KMeans
from sklearn.cluster import Birch
#from sklearn.mixture import GaussianMixture#
#from sklearn.cluster import DBSCAN
from scipy.stats import binned_statistic
import numpy.ma as ma

In [20]:
# Process Data
def getData(fname="", procName="Events"):
    kn_dict = uproot.open(fname)["Events"].arrays()
    kn_dict_ak1 = {name.decode(): ak.from_awkward0(array) for name, array in kn_dict.items()}
    kn_events = ak.zip({"Electrons":ak.zip({
                                            "ge":      kn_dict_ak1["ge"],
                                            "gvx":      kn_dict_ak1["gvx"],
                                            "gvy":      kn_dict_ak1["gvy"],
                                            "gvz":      kn_dict_ak1["gvz"],
                                            "gpx":      kn_dict_ak1["gpx"],
                                            "gpy":      kn_dict_ak1["gpy"],
                                            "gpz":      kn_dict_ak1["gpz"],
                                        }),
                        "Hits":ak.zip({
                                      "detID":   kn_dict_ak1["hit_detID"],
                                      "edep":    kn_dict_ak1["hit_edep"],
                                      "elmID":   kn_dict_ak1["hit_elmID"],
                                      "truthx":  kn_dict_ak1["hit_truthx"],
                                      "truthy":  kn_dict_ak1["hit_truthy"],
                                      "truthz":  kn_dict_ak1["hit_truthz"],
                                      "hit_pos":  kn_dict_ak1["hit_pos"],
                                      }),
                        "St23_Tracklet":ak.zip({
                                        "st23tracklet_x_st1":   kn_dict_ak1["st23tracklet_x_st1"],
                                        "st23tracklet_x_st3":   kn_dict_ak1["st23tracklet_x_st3"],
                                        "st23tracklet_y_st1":   kn_dict_ak1["st23tracklet_y_st1"],
                                        "st23tracklet_y_st3":   kn_dict_ak1["st23tracklet_y_st3"],
                                        }),
                        "track":ak.zip({
                                      "x":   kn_dict_ak1["track_x_CAL"],
                                      "y":    kn_dict_ak1["track_y_CAL"],
                                      "ID":    kn_dict_ak1["eventID"],
                            
                                      }),
                       }, depth_limit=1)
    return kn_events

In [21]:
def emcal_simhit_selection(arr):
    mask = (arr.detID == 100)
    return mask

def emcal_simhit_selection_energy(arr, e):
    mask = (arr.edep >= e)
    return mask

def h2_selection(arr):
    mask = (arr.detID >= 35) & (arr.detID <= 38)
    return mask

def st2_selection(arr):
    mask = (arr.detID >= 13) & (arr.detID <= 18)
    return mask

def st3_selection(arr):
    mask = (arr.detID >= 19) & (arr.detID <= 30)
    return mask

def h4_selection(arr):
    mask = (arr.detID >= 41) & (arr.detID <= 46)
    return mask

In [22]:
ntowersx=72
ntowersy=36
sizex=5.53 # in cm
sizey=5.53 # in cm
ecalx=[-200,200] #size in cm
ecaly=[-100,100]
binsx=ecalx[1]- ecalx[0]
binsy=ecaly[1]- ecaly[0]
sfc = 0.1146337964120158 #sampling fraction of emcal
emin=0.0005


In [23]:
def emcal_bytuple(file):
    dq_events = getData(file,"Events")
    x_pos = []#designed to be 2D
    y_pos = []#designed to be 2D
    eve_energy = []#designed to be 2D
    for i in range(len(dq_events[:]["Hits"].edep)):
        output=emcal_byevent(dq_events, i)
        if(len(output[0])!=0):
            x_pos.append(output[0])
            y_pos.append(output[1])
            eve_energy.append(output[2])
    return x_pos, y_pos, eve_energy

In [24]:
def emcal_byevent(dq_events,evtNum):
    dq_hits = dq_events[evtNum]["Hits"]
    # select emcal hits
    emcal_mask = emcal_simhit_selection(dq_hits)
    emcal_hits = dq_hits[emcal_mask]
    emcal_energy_mask = emcal_simhit_selection_energy(emcal_hits, emin)
    emcal_hits = emcal_hits[emcal_energy_mask]
    #convert into coordinates and energy_dp
    emcal_towerx = emcal_hits.elmID//ntowersy
    emcal_towery = emcal_hits.elmID%ntowersy
    emcal_x = ecalx[0]+emcal_towerx*sizex
    emcal_y = ecaly[0]+emcal_towery*sizey
    emcal_edep = emcal_hits.edep/sfc
    return emcal_x, emcal_y, emcal_edep

In [25]:
def find_lead_clus(label, eve_num, eng_eve):
    count=[0]*(len(np.unique(label)))
    for i in range(len(label)):
        count[label[i]]+=eng_eve[eve_num][i]
    return np.argmax(count)

In [26]:
def Clustering_tuple(file):
    output=emcal_bytuple(file)
    x_eve=output[0]
    y_eve=output[1]
    eng_eve=output[2]
    labels=[]#2D, for all hits
    lead_nums=[]#for each event
    for i in range(len(eng_eve)):
        coordinate=np.column_stack((x_eve[i],y_eve[i]))
        brc = Birch(threshold=20, n_clusters=None)#maximum radius of a cluster is 20, no limit on how much clusters
        brc.fit(coordinate)
        label=brc.predict(coordinate)
        labels.append(label)
        lead_nums.append(find_lead_clus(label, i, eng_eve))
    return labels, lead_nums, x_eve, y_eve, eng_eve

In [27]:
def find_coord(file):
    output=Clustering_tuple(file)
    labels=output[0]
    lead_nums=output[1]
    x_eve=output[2]
    y_eve=output[3]
    eng_eve=output[4]
    new_x=[]
    new_y=[]
    new_eng=[]
    for i in range(len(lead_nums)):
        x_1d=[]
        y_1d=[]
        eng_1d=[]
        for j, k in enumerate(labels[i]):
            if (k==lead_nums[i]):
                x_1d.append(x_eve[i][j])
                y_1d.append(y_eve[i][j])
                eng_1d.append(eng_eve[i][j])
        new_x.append(x_1d)
        new_y.append(y_1d)
        new_eng.append(eng_1d)
    return new_x, new_y, new_eng

In [28]:
def into_layers(file):
    interval=[0, 25, 50, 75, 105]
    output=find_coord(file)
    x_eve=output[0]
    y_eve=output[1]
    eng_eve=output[2]
    new_x=[[], [], [], []]
    new_y=[[], [], [], []]
    new_eng=[[], [], [], []]
    for i in range(len(eng_eve)):
        eng=sum(eng_eve[i])
        if(eng>=interval[2]):
            if(eng>=interval[3]):
                n=3
            else:
                n=2
        else:
            if(eng>=interval[1]):
                n=1
            else:
                n=0
        new_x[n].append(x_eve[i])
        new_y[n].append(y_eve[i])
        new_eng[n].append(eng_eve[i])
    return new_x, new_y, new_eng
            

In [29]:
def gen_wew(file):
    output=into_layers(file)
    x_eve=output[0]
    y_eve=output[1]
    eng_eve=output[2]
    wew_x = [[], [], [], []]
    wew_y = [[], [], [], []]
    for k in range(len(eng_eve)):
        for i in range(len(eng_eve[k])):
            eng_tot=sum(eng_eve[k][i])
            x_bar=(np.dot(x_eve[k][i], eng_eve[k][i]))/eng_tot
            y_bar=(np.dot(y_eve[k][i], eng_eve[k][i]))/eng_tot
            x_sq_eve = []
            y_sq_eve = []
            for j in range(len(eng_eve[k][i])):
                x_sq_eve.append(eng_eve[k][i][j]*(x_eve[k][i][j]-x_bar)**2)
                y_sq_eve.append(eng_eve[k][i][j]*(y_eve[k][i][j]-y_bar)**2)
            try: wew_x[k].append(math.sqrt(sum(x_sq_eve)/eng_tot))
            except ZeroDivisionError: pass
            try: wew_y[k].append(math.sqrt(sum(y_sq_eve)/eng_tot))
            except ZeroDivisionError: pass

    return wew_x, wew_y
            

In [30]:
def gen_wid(file):
    output=into_layers(file)
    x_eve=output[0]
    y_eve=output[1]
    wid_x = [[], [], [], []]
    wid_y = [[], [], [], []]
    for k in range(len(x_eve)):
        for i in range(len(x_eve[k])):
            try:x_bar=sum(x_eve[k][i])/len(x_eve[k][i])
            except ZeroDivisionError: x_bar=0
            try:y_bar=sum(y_eve[k][i])/len(y_eve[k][i])
            except ZeroDivisionError: y_bar=0
            x_sq_eve = []
            y_sq_eve = []
            for j in range(len(x_eve[k][i])):
                x_sq_eve.append((x_eve[k][i][j]-x_bar)**2)
                y_sq_eve.append((y_eve[k][i][j]-y_bar)**2)
            try:wid_x[k].append(math.sqrt(sum(x_sq_eve)/len(x_eve[k][i])))
            except ZeroDivisionError: pass
            try:wid_y[k].append(math.sqrt(sum(y_sq_eve)/len(y_eve[k][i])))
            except ZeroDivisionError: pass
    return wid_x, wid_y

In [31]:
def filter():
    dir_10000="/Users/wongdowling/Desktop/DQ_Dowling/samples"
    dir_1000="/Users/wongdowling/Desktop/DQ_Dowling/ntuple_output_1000"
    os.chdir(dir_1000)
    file1="electron_520_1000eve.root"
    file2="kaon_520_1000eve.root"
    wew_e=gen_wew(file1)
    wew_k=gen_wew(file2)
    wew_x_kaon_electron=[[[0, 1.2],[1.2, 2.8],[2.8, 12.4],[12.4, 20]], 
                         [[0, 2.8], [2.8, 9.2], [9.2, 20]],
                         [[0, 2.8], [2.8, 9.2], [9.2, 20]], 
                         [[0, 0.8], [0.8, 2.4], [2.4, 5.6], [5.6, 20]]
                        ]
    elec_mask=[[False, True, False, True],
               [True, False, True],
               [True, False, True],
               [False, True, False, True]  
    ]
    kaon_mask=[[True, False, True, False],
               [False, True, False],
               [False, True, False],
               [True, False, True, False]  
    ]
    

In [32]:
def count(list1, xl, x2):
    # Using list comprehension to filter elements within the range
    # and returning the length of the resulting list
    return len([x for x in list1 if x >= x1 and x <= x2])

In [33]:
dir_1000="/Users/wongdowling/Desktop/DQ_Dowling/ntuple_output_1000"
os.chdir(dir_1000)
file1="klong_520_1000eve.root"
wew_e=gen_wew(file1)
wew_e_x=wew_e[0]
wew_x_kaon_electron=[[[0, 1.2],[1.2, 2.8],[2.8, 12.4],[12.4, 20]], 
                         [[0, 2.8], [2.8, 9.2], [9.2, 20]],
                         [[0, 2.8], [2.8, 9.2], [9.2, 20]], 
                         [[0, 0.8], [0.8, 2.4], [2.4, 5.6], [5.6, 20]]
                        ]
elec_mask=[[False, True, False, True],
               [True, False, True],
               [True, False, True],
               [False, True, False, True]  
]
empty=[[], [], [], []]
for i in range(len(wew_e_x)):
    arr=np.array(wew_e_x[i])
    for j in range(len(wew_x_kaon_electron[i])):
        empty[i].append(count(arr, wew_x_kaon_electron[i][j][0], wew_x_kaon_electron[i][j][1]))
empty


FileNotFoundError: [Errno 2] No such file or directory: '/Users/wongdowling/Desktop/DQ_Dowling/ntuple_output_1000'

In [31]:
empty
72.8%


[[3, 104, 46, 2], [218, 71, 8], [227, 52, 4], [0, 156, 100, 9]]

In [28]:
n=0

for i in range(len(empty)):
    n+=sum(empty[i])
print(n)

1000


In [39]:
dir_10000="/Users/wongdowling/Desktop/DQ_Dowling/samples"
os.chdir(dir_10000)
file1="st23_kaon_520_10000eve.root"
output=emcal_bytuple(file1)


In [42]:
len(output[2])

5309

In [43]:
dir_1000="/Users/wongdowling/Desktop/DQ_Dowling/ntuple_output_1000"
os.chdir(dir_1000)
file1="klong_520_1000eve.root"
output=emcal_bytuple(file1)
len(output[0])

529

In [34]:
dir_1000="/Users/wongdowling/Desktop/DQ_Dowling/ntuple_big"
os.chdir(dir_1000)
file1="electron_520_200000eve.root"
wew_e=gen_wew(file1)
wew_e_x=wew_e[0]
wew_x_kaon_electron=[[[0, 1.2],[1.2, 2.8],[2.8, 12.4],[12.4, 20]], 
                         [[0, 2.8], [2.8, 9.2], [9.2, 20]],
                         [[0, 2.8], [2.8, 9.2], [9.2, 20]], 
                         [[0, 0.8], [0.8, 2.4], [2.4, 5.6], [5.6, 20]]
                        ]
elec_mask=[[False, True, False, True],
               [True, False, True],
               [True, False, True],
               [False, True, False, True]  
]
empty=[[], [], [], []]
for i in range(len(wew_e_x)):
    arr=np.array(wew_e_x[i])
    for j in range(len(wew_x_kaon_electron[i])):
        empty[i].append(count(arr, wew_x_kaon_electron[i][j][0], wew_x_kaon_electron[i][j][1]))
rate=(empty[0][1]+empty[0][3]+empty[1][0]+empty[1][2]+empty[2][0]+empty[2][2]+empty[3][1]+empty[3][3])/200000
print(rate)

NameError: name 'x1' is not defined

In [ ]:
gen_wid("/Users/wongdowling/Desktop/DQ_Dowling/ntuple_1000/muon_520_1000eve.root")